# Toulouse public library dataset analysis.

> Test your data analysis skills with some real world data processing problems.

#mojibake #datamisalignment

![A photograph of the Toulouse public Library](img/bibliothèque_toulouse.gif "Photographie de la médiathèque José Cabanis, Toulouse (Source: Toulouse Métropole).")

From the shelves of Toulouse's public libraries, this notebook captures a snapshot of its customer habits in an unusual fashion. Whether you're a data enthusiast, a library lover, or simply curious about the urban cultural tastes of the pink city, you're about to follow a data trail and jump into a rabbit hole made of musical, cinematic and literary explorations.

Beware: the raw datasets are flawed, making the path to a usable dataset bumpy. We'll need to address several issues as we go.

The full datasets are available on the [Toulouse Metropolis open data portal](https://data.toulouse-metropole.fr/pages/accueil/) and in this repository for reproducibility purposes.

- [top 500 books loans](https://data.toulouse-metropole.fr/explore/dataset/top-500-des-imprimes-les-plus-empruntes-a-la-bibliotheque-de-toulouse/information/)
- [top 500 movies loans](https://data.toulouse-metropole.fr/explore/dataset/top-500-des-films-les-plus-empruntes-a-la-bibliotheque-de-toulouse/information/)
- [top 500 songs loans](https://data.toulouse-metropole.fr/explore/dataset/top-500-des-cds-les-plus-empruntes-a-la-bibliotheque-de-toulouse/information/)

They are provided under a `Licence Ouverte v2.0 (Etalab)` [license](https://www.etalab.gouv.fr/wp-content/uploads/2017/04/ETALAB-Licence-Ouverte-v2.0.pdf).

> I am not young enough to know everything.
> 
> James M. Barrie

## Table of contents

  Toulouse public library dataset analysis. 
- [Part I – A dataset that needs some patching](#toulouse-public-library-dataset-analysis-part-I--a-dataset-that-(definately)-needs-some-patching)
  - [Steps](#steps)
  - [Requirements](#requirements)
  - [Imports](#imports)
  - [Configuration](#configuration)
  - [Utilities](#utilities)
    - [A quest for the first matching path](#a-quest-for-the-first-matching-path)
    - [Why encoding detection matters](#why-encoding-detection-matters)
    - [Broken alignments: fixing the lines](#broken-alignments-fixing-the-lines)
- [Part II – Data analysis](#part-ii--data-analysis)
  - [The LoanAnalysis Class: overview](#the-loananalyis-class-overview)
  - [Trending and shooting star logics](#trending-and-shooting-star-logics)
  - [Main orchestration](#main-orchestration)
- [References](#references)

---

# Part I - A dataset that (definately) needs some patching

The dataset consists of 3 homogeneous CSV files (with identical column structures). One per category:
- Prints
- Movies
- Music

| Column name   | Type    | Description                     | Example      | 
|---------------|---------|---------------------------------|--------------|
| ANNEE         | Integer | Year of the record              | 2019         | 
| Nbre de prêts | Integer | Number of loans                 | 93           |
| TITRE         | String  | Title of the work               | Enfantillages|
| AUTEUR        | String  | Author                          | Aldebert     | 
| Editeur       | String  | Publisher                       | Skyzo Music | 
| Indice        | String  | Index                 | S099.2            | 
| BIB           | String  | Library code                    | CABANIS       | 
| COTE          | String  | Location label    | E 780.2 ALD     | 
| Cat 1         | String  | Category label 1 (Audience)                | E       | 
| Cat 2         | String  | Category label 2    (Media type)            | CD          | 

- “–” represents missing data

## Steps

**Data processing:**
- Load CSV files matching discriminators (e.g. contains the "*imprimes*" (prints) keyword).
- Detect file encoding and fix *mojibake* characters.
- Repair line-level misalignment.

**Data analysis:** 
- Aggregate and compute basic insights: top-N, trends (progressions/regressions), and sudden drops in popularity.
- Exports insights in per-discriminator named JSON outputs.

## Requirements
- It is recommended to use Python 3.10+ with jupyterlab 4.4.4+
- If not already in your environment, install pandas and chardet used in the notebook (uncomment the cell below).

In [1]:
# Run once if packages aren't installed
#!pip install pandas chardet

---

## Imports

In [2]:
from dataclasses import dataclass
from typing import Tuple, Mapping, Dict, Iterable, List, Optional
import os
from io import StringIO
from pathlib import Path
import glob
import json
from collections import Counter, defaultdict
import chardet
import pandas as pd

## Configuration

A config dataclass is a Python configuration object using the `@dataclass` decorator to create an immutable (frozen) configuration container for storing settings and parameters. The `Config` class contains keywords to load the expected files and column names from the CSV.

In [3]:
@dataclass(frozen=True)
class Config:
    dataset_folder: str = "dataset"
    output_folder: str = "output"
    discriminator: str = "imprimes"
    discriminator_list: Tuple[str, ...] = ("films", "imprimes", "cds")
    top_n: int = 10
    output_json: str = ""
    year_field: str = "ANNEE"
    popularity_field: str = "Nbre de prêts"
    title_field: str = "TITRE"
    author_field: str = "AUTEUR"
    cat1_field: str = "Cat 1"
    cat1_selector: Tuple[str, ...] = ("A", "E")
    composite_field: str = "Item_ID"
    replacements: Mapping[str, str] = None

    def __post_init__(self):
        object.__setattr__(self, "output_json", f"results_{self.discriminator}.json")
        if self.replacements is None:
            object.__setattr__(self, "replacements", {
                # Empiric replacements observed in some mojibake cases
                'ãa': 'â', 'ãe': 'ê', 'ãi': 'î', 'ão': 'ô',
                'ãu': 'û', 'âe': 'é', 'áa': 'à', 'áe': 'è',
                'ðc': 'ç'
            })

About the **replacements** mapping: 

This is a pragmatic mapping (where both keys and values are strings) for the garbled digraphs observed in the dataset. In this case, it's better to have a *heuristic* approach: inspect real errors and add targeted mappings rather than a general brute-force transform.

## Utilities

### A quest for the first matching path

Before loading the datasets, we need to locate the correct files using wildcard patterns and ensure at least one matching file exists.

In [4]:
def find_first_matching_file(pattern: str) -> str:
    """Return the first file matching glob pattern or raise FileNotFoundError."""
    matches = glob.glob(pattern)
    if not matches:
        raise FileNotFoundError(f"No file matching pattern: {pattern!r}")
    return matches[0]

### Why encoding detection matters

While inspecting the dataset, I ran into encoding issues, these are often overlooked by teachers. Most course's datasets are clean, leading to a dismissal of the problem as lacking real Computer Science value. It's important to be aware of such issues: you will encounter them.

**In short:** 
If you open bytes with the wrong encoding, you get garbled text where accented characters become sequences like `Ã©` instead of `é`. This is commonly refered to as [mojibake](https://en.wikipedia.org/wiki/Mojibake).

> Example: the two-byte UTF-8 encoding for `é` is `0xC3` `0xA9`. If these bytes are interpreted as Latin‑1, they become `Ã` (0xC3) followed by `©` (0xA9) — rendering as `Ã©`.

Character encodings map bytes to characters. CSV files produced by different systems (Windows, Mac, Linux; different locales) may use different encodings (*UTF-8, ISO-8859-1 a.k.a. Latin-1, Windows-1252, CP850, etc.*) that are not interoperable.

Detecting and handling encoding is important when:
- The CSV contains accented characters (French, Spanish, Portuguese..).
- The CSV is exported from legacy systems (often Windows‑1252 or ISO‑8859‑1).
- You need to preserve text for grouping/joins (e.g., the same title must match).

[chardet](https://pypi.org/project/chardet/) guesses the encoding from a byte sample and open the file using that guess. chardet is heuristic — it often guesses correctly but not always. If you already know the encoding from source, prefer that.

*References:*
- Explainer on [encoding issues](https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/)


In [5]:
def sniff_encoding_and_show(filepath: str, sample_bytes: int = 200):
    """Show raw bytes and decode attempts to help choose the correct encoding."""
    dataset_folder = Config.dataset_folder
    filepath = os.path.join(dataset_folder, filepath)
    b = Path(filepath).read_bytes()[:sample_bytes]
    print("Raw bytes (hex):", b.hex())
    guess = chardet.detect(b)
    print("chardet guess:", guess)
    for enc in ("utf-8", "windows-1252", "iso-8859-1", "cp850"):
        try:
            print(f"\n== decode using {enc} ==")
            print(b.decode(enc))
        except Exception as e:
            print(f"Failed to decode with {enc}: {e}")

This example using the function above give you a clear look at a file's byte content and the different ways it might be encoded.

In [6]:
# Example: show encoding diagnostic for a specific file
sniff_encoding_and_show("top-500-des-films-les-plus-empruntes-a-la-bibliotheque-de-toulouse.csv")

Raw bytes (hex): efbbbf414e4e45453b4e627265206465207072c3aa74733b54495452453b4155544555523b456469746575723b496e646963653b4249423b434f54453b43617420313b43617420320d0a323031393b3236333b486172727920506f74746572206574206c61206368616d6272652064657320736563726574733b436f6c756d6275732c2043687269733b2d3b5061726973203a205761726e657220486f6d6520566964656f2c20323030332e3b484152523b434142414e49533b462048415252202f323b450d0a32
chardet guess: {'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}

== decode using utf-8 ==
﻿ANNEE;Nbre de prêts;TITRE;AUTEUR;Editeur;Indice;BIB;COTE;Cat 1;Cat 2
2019;263;Harry Potter et la chambre des secrets;Columbus, Chris;-;Paris : Warner Home Video, 2003.;HARR;CABANIS;F HARR /2;E
2

== decode using windows-1252 ==
ï»¿ANNEE;Nbre de prÃªts;TITRE;AUTEUR;Editeur;Indice;BIB;COTE;Cat 1;Cat 2
2019;263;Harry Potter et la chambre des secrets;Columbus, Chris;-;Paris : Warner Home Video, 2003.;HARR;CABANIS;F HARR /2;E
2

== decode using iso-8859-1 ==
ï»¿ANNEE;Nbre de prÃª

In [7]:
def detect_encoding(filepath: str, sample_bytes: int = 200) -> str:
    """Guess encoding using chardet from a small sample of the file.
    Returns a string encoding; default to 'utf-8' if guess is None."""
    with open(filepath, "rb") as f:
        raw = f.read(sample_bytes)
    result = chardet.detect(raw)
    return result.get("encoding") or "utf-8"

In [8]:
def apply_replacements(val: object, repl: Mapping[str, str]) -> object:
    """Apply a set of string replacements to correct systematic mojibake
    fragments that survived decoding (e.g., 'ãa' -> 'â'). Non-strings left unchanged."""
    if not isinstance(val, str):
        return val
    for wrong, right in repl.items():
        val = val.replace(wrong, right)
    return val

- **Detection strategy:**
  - Use chardet on a bytes sample: gives a best-guess encoding with confidence (`detect_encoding()`).
  - Open file using that encoding and `errors="replace"` (see in the later `load_and_clean_csv()` function) to avoid crashes if there are undecodable bytes (this replaces undecodable bytes with �).
  - Then apply targeted replacements for known garbled sequences (the `Config.replacements` map is called by `apply_replacements()`). This map is an empiric fix for observed corruptions where some compound sequences map to a single accented character (e.g., `ãa` -> `â`).

- Why targeted replacements (not blind re-encoding)? Because:
  - Re-decoding already-read text is messy; it's better to open with correct encoding when possible.
  - Some mojibake arises from double-encoding (bytes were UTF-8 encoded twice) or from search/replace bugs in the source; targeted fixes are safer than attempting general re-transcoding.

### Broken alignments: fixing the lines

A **critical** issue found in the data is row misalignment.

Let's have a look at these two rows:

| ANNEE | Nbre de prêts | TITRE | AUTEUR | Editeur | Indice | BIB   | COTE                                 | Cat 1 | Cat 2 |
|------|---------------|-------|--------|---------|--------|-------|--------------------------------------|-------|-------|
| 2019 | 698           | Okapi | -      | **-**       | **Paris : Bayard presse, 1971-** | **P1468** | **CABANIS** | **P OKAP** | **E** |
| 2022 | 404           | Lire  | -      | Paris : Lire, 1975- | P1225 | CABANIS | P LIRE NO. 420 NOV 2013 | A | PERIO |

The first one (2019) is misaligned. An extra `-` has been inserted in the *Editeur* column, shifting **Paris : Bayard presse, 1971-** (The "Editeur" (Publisher)) and latter values one column to the right.

- **Pattern:** In the datasets, some CSV lines have an extra sequence like `;-;` inserted before the last field. This produces extra empty values or shifts columns right when parsed by pandas, causing misalignment. It is also true that in misaligned rows, the last **character** is a known audience category letter (`A/E/B/P`), these letters are valuable *sentinels*.

- Why did this happen?
Cannot tell for sure. Export scripts sometimes include stray delimiters, escape sequences, or an extra token ("-") inserted into the field separator. It can result from manual concatenation, export bugs, or characters that the export tool tried to escape badly.

In [9]:
def fix_line(line: str) -> str:
    """Small heuristic to correct a known misalignment pattern:
    some exports include the sequence ';-;' immediately before a trailing Cat1
    letter (A/E/B/P). This replaces the first ';-;' with ';' when found and the
    last character is in the expected set. This is targeted, not general."""
    s = line.rstrip()
    if s and s[-1] in {"A", "E", "B", "P"} and ";-;" in s:
        s = s.replace(";-;", ";", 1)
    return s

- How to fix it :
  1. Read the file as raw lines.
  2. For each data line (header excluded), check a simple heuristic: if the line ends with one of the expected Cat1 letters (`"A", "E", "B", "P"`) and contains the pattern `';-;'`, replace the first `';-;'` with `';'`. This repairs the common pattern that caused a one-column shift.
  3. After line repair, parse CSV normally.

- Caveats:
  - This is a targeted heuristic. **It corrects a concrete, observed bug pattern**. Do not use it blindly for all files.
  - If misalignment is caused by quotes or embedded delimiters (e.g., title includes a semicolon not quoted), always prefer fixing the source or using a stricter CSV quoting handler (pandas `read_csv` supports *quotechar* and *escapechar*). For example, if semicolons are legitimately inside quoted fields, ensure the CSV generator quotes them properly; otherwise, write a quoting-aware parser.

Reference on [CSV quoting](https://docs.python.org/3/library/csv.html).

---

# Part II - Data analysis

![A photograph of the Toulouse public Library](img/Médiathèque_José_Cabanis_Toulouse.jpg "Photographie de l'intérieur de la médiathèque José Cabanis, Toulouse (Source Buffi Associés).")

## The LoanAnalysis Class: overview

This Class encapsulates:

- Loading and cleaning CSV files (encoding detection, line fixes, characters replacements).
- Creating a composite key (title + author) to group identical items.
- Filtering on Cat 1 values (cfg.cat1_selector).
- Aggregations to compute top-N counts.
- Pivot construction for per-year time series for each composite item.
- Attaches the Cat 1 value to a record based on the composite key.
- Updates a list of records with their corresponding Cat 1 values
- Flattens nested lists or dictionaries into a single list of records.

In [10]:
class LoanAnalysis:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.cat1_map: Dict[str, str] = {}

    def load_and_clean_csv(self, filepath: str, encoding: Optional[str] = None) -> pd.DataFrame:
        enc = encoding or detect_encoding(filepath)
        # Read raw lines so we can apply line-level fixes before parsing CSV
        with open(filepath, "r", encoding=enc, errors="replace") as fin:
            lines = fin.readlines()
        if not lines:
            raise ValueError("Empty CSV file")

        # Keep header as-is; fix only subsequent lines (targeted repair)
        header = lines[0].rstrip("\n\r")
        data_lines = [fix_line(l) for l in lines[1:]]
        unified = StringIO("\n".join([header] + data_lines))

        # Use pandas to parse; delimiter is ';' for the CSVs
        df = pd.read_csv(
            unified,
            delimiter=";",
            engine="python",
            encoding=enc,
            encoding_errors="replace"
        )

        # Apply character replacements on all object columns
        for c in df.select_dtypes(include=["object"]).columns:
            df[c] = df[c].apply(lambda v: apply_replacements(v, self.cfg.replacements))

        # Normalize numerical columns with safe coercion
        df[self.cfg.year_field] = pd.to_numeric(df[self.cfg.year_field], errors="coerce").astype("Int64")
        df[self.cfg.popularity_field] = pd.to_numeric(df[self.cfg.popularity_field], errors="coerce").astype("Int64")

        # Composite key for items: title + author
        df[self.cfg.composite_field] = (
            df[self.cfg.title_field].astype(str)
            + " --- "
            + df[self.cfg.author_field].astype(str)
        )

        # Build mapping from composite key -> Cat1 (join-back later)
        grp = df.groupby(self.cfg.composite_field)[self.cfg.cat1_field].unique()
        self.cat1_map = {
            k: (", ".join(sorted(v)) if len(v) > 1 else v[0])
            for k, v in grp.to_dict().items()
        }

        # Filter rows by Cat 1 allowed values (cfg.cat1_selector)
        df = df[df[self.cfg.cat1_field].isin(self.cfg.cat1_selector)]

        return df

    def aggregate_by(self, df: pd.DataFrame, groupby_fields: Iterable[str], key_field: str,
                     value_field: str) -> pd.DataFrame:
        fields = list(groupby_fields) + [key_field]
        return df.groupby(fields, as_index=False)[value_field].sum()

    def top_n_from_agg(self, agg_df: pd.DataFrame, value_field: str, top_n: int) -> pd.DataFrame:
        return agg_df.sort_values(value_field, ascending=False).head(top_n)

    def group_top_n(self, df: pd.DataFrame, groupby_fields: Iterable[str],
                    key_field: str, value_field: str, top_n: int) -> Dict[str, List[dict]]:
        agg = self.aggregate_by(df, groupby_fields, key_field, value_field)
        result: Dict[str, List[dict]] = {}
        gb = list(groupby_fields)
        for group_vals, sub in agg.groupby(gb):
            key = ", ".join(map(str, group_vals)) if isinstance(group_vals, tuple) else str(group_vals)
            result[key] = self.top_n_from_agg(sub, value_field, top_n).to_dict(orient="records")
        return result

    def _build_pivot(self, df: pd.DataFrame, index: str, columns: str, values: str) -> pd.DataFrame:
        return df.pivot_table(index=index, columns=columns, values=values, aggfunc="sum", fill_value=0)

    def attach_cat1(self, record: dict) -> dict:
        item = record.get(self.cfg.composite_field)
        if item in self.cat1_map:
            record[self.cfg.cat1_field] = self.cat1_map[item]
        return record

    def update_records_with_cat(self, records: Iterable[dict]) -> List[dict]:
        return [self.attach_cat1(dict(r)) for r in records]

    @staticmethod
    def flatten_top_lists(input_data) -> List[dict]:
        flat: List[dict] = []
        if isinstance(input_data, list):
            flat.extend(input_data)
        elif isinstance(input_data, dict):
            for v in input_data.values():
                if isinstance(v, list):
                    flat.extend(v)
                elif isinstance(v, dict):
                    flat.extend(LoanAnalysis.flatten_top_lists(v))
        return flat

## Trending and shooting star logics

1) **Trending:** finding items whose counts change between their first and last non-zero years in a year-by-year pivot table.

**Steps:**
- `compute_trends()` constructs a pivot table: index = composite item, columns = years, values = loan counts.
- It filters out items that appear in fewer than two non zero years (`pivot_df > 0).sum(axis=1) >= 2`) because progression/regression requires at least two data points.
- For each item, it finds the first and last year with non-zero loans. It then compares loan counts in those years: diff = last - first.
  - progression: `diff > 0` (increasing popularity).
  - regression: `diff < 0` (decreasing popularity).
  
- `compute_trend_by_group()` splits a DataFrame by a grouping field, builds a pivot for each group, calls `compute_trends` and collects the per-group trend results as lists of dictionaries.

In [11]:
def compute_trends(self, pivot_df: pd.DataFrame, trend: str = "progression", top_n: Optional[int] = None) -> pd.DataFrame:
    top_n = top_n or self.cfg.top_n
    mask = (pivot_df > 0).sum(axis=1) >= 2
    pivot_filtered = pivot_df[mask].copy()
    if pivot_filtered.empty:
        return pd.DataFrame(columns=[self.cfg.composite_field, "first_year", "last_year", "sum_first", "sum_last", "diff"])

    # first and last non-zero year (columns are years; ensure they're ints)
    first_year = pivot_filtered.apply(lambda r: int(r[r > 0].index.min()), axis=1)
    last_year = pivot_filtered.apply(lambda r: int(r[r > 0].index.max()), axis=1)

    sum_first = []
    sum_last = []
    for idx, fy, ly in zip(pivot_filtered.index, first_year, last_year):
        sum_first.append(int(pivot_filtered.at[idx, fy]))
        sum_last.append(int(pivot_filtered.at[idx, ly]))

    df_trend = pd.DataFrame({
        self.cfg.composite_field: pivot_filtered.index,
        "first_year": first_year.values,
        "last_year": last_year.values,
        "sum_first": sum_first,
        "sum_last": sum_last,
        "diff": [s_l - s_f for s_f, s_l in zip(sum_first, sum_last)]
    }).reset_index(drop=True)

    if trend == "progression":
        return df_trend[df_trend["diff"] > 0].sort_values("diff", ascending=False).head(top_n)
    if trend == "regression":
        return df_trend[df_trend["diff"] < 0].sort_values("diff", ascending=True).head(top_n)
    raise ValueError("trend must be 'progression' or 'regression'.")

def compute_trend_by_group(self, df: pd.DataFrame, group_field: str, value_field: str, trend: str) -> Dict[str, List[dict]]:
    result: Dict[str, List[dict]] = {}
    for grp, sub in df.groupby(group_field):
        pivot = self._build_pivot(sub, self.cfg.composite_field, self.cfg.year_field, value_field)
        trend_df = self.compute_trends(pivot, trend=trend)
        for col in ("sum_first", "sum_last", "diff"):
            if col in trend_df:
                trend_df[col] = trend_df[col].astype(int)
        result[str(grp)] = trend_df.to_dict(orient="records")
    return result

2) **Disappearing:** detect items that suddenly drop to zero the year after their last recorded non-zero year in a year-by-year pivot table.
   
**Steps:**

- `compute_sudden_disappearances()` iterates each item, finds its non-zero years, takes the last such year, and if the immediate next year exists in the columns with a zero value it records the item, that last year, and the value in that last year. 

*In short:* finds items with loans in some year Y and zero loans in Y+1 (explicitly checks existence of Y+1 column). This is a pragmatic definition of a shooting star.
- `compute_disappearances_by_group()` groups the input DataFrame, builds a year pivot for each group, runs the disappearance detector, and returns per-group lists of disappearance records.

In [12]:
def compute_sudden_disappearances(self, pivot_df: pd.DataFrame, top_n: Optional[int] = None) -> pd.DataFrame:
    top_n = top_n or self.cfg.top_n
    records = []
    for item in pivot_df.index:
        # columns are years; find non-zero years for this item
        nonzero = pivot_df.columns[pivot_df.loc[item] > 0]
        if len(nonzero) == 0:
            continue
        last_year = nonzero.max()
        # check if next year exists and is zero (sudden disappearance)
        if (last_year + 1) in pivot_df.columns and pivot_df.loc[item, last_year + 1] == 0:
            records.append({
                self.cfg.composite_field: item,
                "last_year": int(last_year),
                "loan_last": int(pivot_df.loc[item, last_year])
            })
    if not records:
        return pd.DataFrame(columns=[self.cfg.composite_field, "last_year", "loan_last"])
    return pd.DataFrame(records).sort_values("loan_last", ascending=False).head(top_n)

def compute_disappearances_by_group(self, df: pd.DataFrame, group_field: str, value_field: str) -> Dict[str, List[dict]]:
    result: Dict[str, List[dict]] = {}
    for grp, sub in df.groupby(group_field):
        pivot = self._build_pivot(sub, self.cfg.composite_field, self.cfg.year_field, value_field)
        df_disp = self.compute_sudden_disappearances(pivot)
        result[str(grp)] = df_disp.to_dict(orient="records")
    return result

- `run_all()` builds a comprehensive report of "top" lists and trend analyses from a DataFrame. It:

  - Aggregates overall popularity and returns the top N overall.
  - Computes top N per year (merged across audiences) and top N separated by audience type.
  - Counts which titles appear most often among yearly top-N lists (overall and per audience).
  - Computes progressions and regressions per audience and overall by building a year-by-year pivot and using the trend functions.
  - Detects shooting stars (sudden disappearances) per audience and overall.
  - Tallies nominations across many result lists to produce the most frequently nominated titles.

- Returns a dictionary R whose keys describe each result section; each value contains a short question string and the resulting records (lists or dicts) ready for downstream use.

In [13]:
def run_all(self, df: pd.DataFrame) -> Dict[str, dict]:
    R = {}
    C = self.cfg

    # topN overall across all years (aggregated)
    agg_all = self.aggregate_by(df, [], C.composite_field, C.popularity_field)
    topN_all = self.top_n_from_agg(agg_all, C.popularity_field, C.top_n).to_dict(orient="records")
    R["topN_all_years_all_cat1"] = {
        "question": f"Top {C.top_n} most popular items overall",
        "result": self.update_records_with_cat(topN_all)
    }

    # Top per year (merged across Cat1)
    agg_year = self.aggregate_by(df, [C.year_field], C.composite_field, C.popularity_field)
    by_year: Dict[int, List[dict]] = {}
    for yr, sub in agg_year.groupby(C.year_field):
        recs = self.top_n_from_agg(sub, C.popularity_field, C.top_n).to_dict(orient="records")
        by_year[int(yr)] = self.update_records_with_cat(recs)
    R["topN_by_year_all_cat1"] = {
        "question": f"Top {C.top_n} most popular items per year, all audiences merged",
        "result": by_year
    }

    # Tops separated by Cat1
    R["topN_all_years_separated_cat1"] = {
        "question": f"Top {C.top_n} most popular items across all years, categorized by audience (A: adults, E: kids)",
        "result": self.group_top_n(df, [C.cat1_field], C.composite_field, C.popularity_field, C.top_n)
    }
    R["topN_by_year_separated_cat1"] = {
        "question": f"Top {C.top_n} most popular items per year, categorized by audience (A: adults, E:kids)",
        "result": self.group_top_n(df, [C.year_field, C.cat1_field], C.composite_field, C.popularity_field, C.top_n)
    }

    # Titles appearing most often across yearly topNs (merged)
    merged_counter = Counter()
    for recs in R["topN_by_year_all_cat1"]["result"].values():
        for rec in recs:
            merged_counter[rec[C.composite_field]] += 1
    R["topN_titles_occurring_most_all_cat1"] = {
        "question": f"Top {C.top_n} titles that appear most frequently among top tens (all audiences)",
        "result": [
            {C.composite_field: k, "appearances": v, C.cat1_field: self.cat1_map.get(k, "")}
            for k, v in merged_counter.most_common(C.top_n)
        ]
    }

    # Titles appearing most often per Cat1
    temp_counter: Dict[str, Counter] = defaultdict(Counter)
    for group_key, recs in R["topN_by_year_separated_cat1"]["result"].items():
        cat = group_key.split(",")[-1].strip() if "," in group_key else group_key
        for rec in recs:
            temp_counter[cat][rec[C.composite_field]] += 1

    sep_occ: Dict[str, List[dict]] = {}
    for cat, counter in temp_counter.items():
        sep_occ[cat] = [
            {C.composite_field: k, "appearances": v, C.cat1_field: cat}
            for k, v in counter.most_common(C.top_n)
        ]
    R["topN_titles_occurring_most_separated_cat1"] = {
        "question": f"Top {C.top_n} titles that appear most frequently among top tens (categorized by audiences)",
        "result": sep_occ
    }

    # Progressions/regressions and disappearances
    R["topN_progressions_separated_cat1"] = {
        "question": f"Top {C.top_n} progressions, categorized by audience",
        "result": self.compute_trend_by_group(df, C.cat1_field, C.popularity_field, trend="progression")
    }

    pivot_all = self._build_pivot(df, C.composite_field, C.year_field, C.popularity_field)
    prog_all = self.compute_trends(pivot_all, trend="progression")
    prog_all = prog_all.astype({"sum_first": int, "sum_last": int, "diff": int})
    R["topN_progressions_all_cat1"] = {
        "question": f"Top {C.top_n} progressions, overall",
        "result": self.update_records_with_cat(prog_all.to_dict(orient="records"))
    }

    R["topN_regressions_separated_cat1"] = {
        "question": f"Top {C.top_n}, downward trend, categorized by audience",
        "result": self.compute_trend_by_group(df, C.cat1_field, C.popularity_field, trend="regression")
    }

    reg_all = self.compute_trends(pivot_all, trend="regression")
    reg_all = reg_all.astype({"sum_first": int, "sum_last": int, "diff": int})
    R["topN_regressions_all_cat1"] = {
        "question": f"Top {C.top_n} downward trend, overall",
        "result": self.update_records_with_cat(reg_all.to_dict(orient="records"))
    }

    R["topN_sudden_disappearances_separated_cat1"] = {
        "question": f"Top {C.top_n} shooting stars categorized by audience",
        "result": self.compute_disappearances_by_group(df, C.cat1_field, C.popularity_field)
    }

    disp_all = self.compute_sudden_disappearances(pivot_all)
    R["topN_sudden_disappearances_all_cat1"] = {
        "question": f"Top {C.top_n} shooting stars, overall",
        "result": self.update_records_with_cat(disp_all.to_dict(orient="records"))
    }

    # Nomination counting across many result lists to find frequently nominated titles
    keys = [
        "topN_all_years_all_cat1",
        "topN_by_year_all_cat1",
        "topN_all_years_separated_cat1",
        "topN_by_year_separated_cat1",
        "topN_sudden_disappearances_all_cat1",
        "topN_sudden_disappearances_separated_cat1",
        "topN_titles_occurring_most_separated_cat1",
        "topN_progressions_all_cat1",
        "topN_progressions_separated_cat1",
        "topN_regressions_all_cat1",
        "topN_regressions_separated_cat1"
    ]
    nom_counter = Counter()
    for k in keys:
        data = R.get(k, {}).get("result", {})
        for rec in self.flatten_top_lists(data):
            nom_counter[rec[C.composite_field]] += 1

    topN_nominated = [
        {C.composite_field: k, "nominations": v, C.cat1_field: self.cat1_map.get(k, "")}
        for k, v in nom_counter.most_common(C.top_n)
    ]
    R["topN_most_nominated_entries"] = {
        "question": f"Top {C.top_n} most nominated entries",
        "result": topN_nominated
    }

    return R

## Main orchestration

In [14]:
# attach orphan functions to LoanAnalysis
LoanAnalysis.compute_trends = compute_trends
LoanAnalysis.compute_sudden_disappearances = compute_sudden_disappearances
LoanAnalysis.compute_trend_by_group = compute_trend_by_group
LoanAnalysis.compute_disappearances_by_group = compute_disappearances_by_group
LoanAnalysis.run_all = run_all

- Re-creates a Config for each discriminator (so output file names differ).
- `detect_encoding()` is called before reading to make an informed choice.
- Outputs are saved with UTF-8 to ensure consistent JSON encoding.

In [15]:
def main(cfg: Optional[Config] = None) -> None:
    cfg = cfg or Config()
    for disc in cfg.discriminator_list:
        local_cfg = Config(
            dataset_folder=cfg.dataset_folder,
            output_folder=cfg.output_folder,
            discriminator=disc,
            discriminator_list=cfg.discriminator_list,
            top_n=cfg.top_n,
            year_field=cfg.year_field,
            popularity_field=cfg.popularity_field,
            title_field=cfg.title_field,
            author_field=cfg.author_field,
            cat1_field=cfg.cat1_field,
            cat1_selector=cfg.cat1_selector,
            composite_field=cfg.composite_field,
            replacements=cfg.replacements
        )

        pattern = f"*{local_cfg.discriminator}*.csv"
        try:
            filepath = find_first_matching_file(os.path.join(local_cfg.dataset_folder, pattern))
        except FileNotFoundError:
            print(f"No CSV found for discriminator {local_cfg.discriminator!r} (pattern: {pattern}). Skipping.")
            continue

        enc = detect_encoding(filepath)
        print(f"[{local_cfg.discriminator}] Detected encoding: {enc}")

        analysis = LoanAnalysis(local_cfg)
        df = analysis.load_and_clean_csv(filepath, encoding=enc)
        
        results = analysis.run_all(df)

        output_filepath = os.path.join(local_cfg.output_folder, "results_" + local_cfg.discriminator + ".json")
        # Create output folder if it doesn't exist
        os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
        with open(output_filepath, "w", encoding="utf-8") as fout:
            json.dump(results, fout, ensure_ascii=False, indent=4)
        print(f"Wrote results to {output_filepath}")

In [16]:
# Run the full pipeline
main()

[films] Detected encoding: UTF-8-SIG


Wrote results to output/results_films.json
[imprimes] Detected encoding: UTF-8-SIG
Wrote results to output/results_imprimes.json
[cds] Detected encoding: UTF-8-SIG
Wrote results to output/results_cds.json


### References
- [chardet](https://pypi.org/project/chardet/)
- [Unicode, encodings and mojibake (Joel Spolsky)](https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/)
- [Mojibake (Wikipedia)](https://en.wikipedia.org/wiki/Mojibake)
- [Python csv module](https://docs.python.org/3/library/csv.html)
- [pandas read_csv docs](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) (useful parameters: encoding, sep, delimiter, quotechar, escapechar, engine): 

Big thanks to the [Toulouse métropole open data team](https://data.toulouse-metropole.fr/page/opendata/) and the library webmaster for sharing a dataset that reveals Toulouse’s cultural heartbeat.

**Note:** if you prefer not to use this interactive jupyter notebook, you can run the same code as a standalone Python script. The standalone script is included in the repository.

![Outside photographs of the Toulouse public Library](img/toulouse_public_library.jpg "Photographies de l'intérieur de la médiathèque José Cabanis, Toulouse (Source Buffi Associés).")

**Facts about the Toulouse main public library:**

Médiathèque José Cabanis

The building features a blend of modern, austere lines and various materials, combining the warmth of terracotta and wood with the brightness of glass and metal. Its forecourt, designed by Guy de Rougemont, consists of gray, black, white, and pink granite slabs. The rear façade is brick-clad, while the front is primarily glass, wood, accented by terracotta sunshades that provide protection without blocking views. These panels also regulate temperature and according to Jean-Pierre Buffi, they also create a light filter for the more intimate interior spaces, echoing inside the city's red brick architecture.

Inside, a vertical shaft spans the first four floors and basement, enhancing the building's height. A large spiral staircase made of metal blades connects the levels, emphasizing verticality. Public areas are organized around a central void that offers panoramic views, complemented by a panoramic elevator showcasing the district's tiled roofs. The roof, designed as a mesh, serves as a fifth façade visible from Jolimont Hill. The garden level opens to a landscaped English courtyard with brick slabs arranged in a herringbone pattern.

- Médiathèque/auditorium : 13 500 m2 
- Institut national de l'audiovisuel (INA) : 1000 m2 
- Reception spaces : 2500 m2 
- Cafés and shops : 1300 m2 
- Local TV 'TLT : 1300 m2 
- Parking and train/metro access: 2000 m2

  - Total surface : 25 500 m2 (I know, it's not adding up -no idea how it's calculated :)

    - Finished building on april 2004 after 6 years of works.
    - Costs: 35,02 M€
    - Architects : Jean-Pierre et Marianne Buffi avec F. E. Greteau et C. Ramin / associates : Séquences Architectes, J. Hurtevent et P. Laborderie

**Opening hours** (find me wandering there at times!):

  - Tuesday: 10am–7pm
  - Wednesday: 10am–7pm
  - Thursday: 2pm–7pm
  - Friday: 2pm–7pm
  - Saturday: 10am–7pm
  - Sunday: 2pm–6pm 

Completed in september 2025 by [brk♛](github.com/brooks-code).

License: ["CC0 - No rights reserved"](https://creativecommons.org/public-domain/cc0/)